#Material Genome 

The code below reads in the original "melting points.txt" document and extract all binary metal systems. This was required as a list of material systems was required to make the api calls to the server.

In [26]:
import os
import collections
import json
import requests

from itertools import izip
import warnings
warnings.filterwarnings('ignore')

# Read in the file and exxtract every entry in the dataset:
INPUT_FILE_NAME = "melting_points copy.txt"

input_data = open(INPUT_FILE_NAME)
element_systems = []
for line in input_data:
    line = line.strip().split(' ',1)
    if (len(line) > 1) and ("-" in line [0]):
        element_systems.append(line[0])
    else: 
        continue
input_data.close()

element_systems = set(element_systems)

The Below cell extracts all sub-directory names. this is required as a list of properties required is to be passed on to the API for each material. (Requires cleaning, this gives some unwanted properties also)

In [ ]:
# Read names of all files in the sub-directory /mapidocs/materials (forked form github)

features = next(os.walk('./mapidoc/materials'))[1]

Funtion to flatten the resulting dict of dicts in the API call(stackoverflow : http://stackoverflow.com/questions/6027558/flatten-nested-python-dictionaries-compressing-keys). 
Eg output: 

 `flatten({'a': 1, 'c': {'a': 2, 'b': {'x': 5, 'y' : 10}}, 'd': [1, 2, 3]})`

{'a': 1, 'c_a': 2, 'c_b_x': 5, 'd': [1, 2, 3], 'c_b_y': 10}

In [ ]:
def flatten(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

This Code should send repeated calls to the server and extract all information (Incomplete)

In [ ]:
for e in element_systems:
    data = {
    'criteria': {"chemsys" : e},
    'properties': features
    }
    r = requests.post('https://materialsproject.org/rest/v2/query',
                 headers={'X-API-KEY': 'exmWCtPDgcFoSXOi '},
                 data={k: json.dumps(v) for k,v in data.iteritems()})
    response_content = r.json() # a dict
    for item in response_content['response']:
        
    

## Examples for One system "Al-LU"

Example of one system and corresponding outpus below

In [29]:
data = {
    'criteria': {"chemsys" : "Al-Lu", "nelements" : 2},
    'properties': features
}
r = requests.post('https://materialsproject.org/rest/v2/query',
                 headers={'X-API-KEY': 'exmWCtPDgcFoSXOi '},
                 data={k: json.dumps(v) for k,v in data.iteritems()})
response_content = r.json() # a dict
for item in response_content['response']

In [30]:
response_content['response'] # need to figure out a way to extract information from this 

[{u'anonymous_formula': {u'A': 1.0, u'B': 3.0},
  u'band_gap': 0.07670000000000066,
  u'band_structure': None,
  u'band_structure_uniform': None,
  u'blessed_tasks': {u'GGA optimize structure (2x)': u'mp-2805',
   u'GGA static v2': u'mp-911553'},
  u'bv_structure': {u'@class': u'Structure',
   u'@module': u'pymatgen.core.structure',
   u'lattice': {u'@class': u'Lattice',
    u'@module': u'pymatgen.core.lattice',
    u'a': 4.20439992,
    u'alpha': 90.0,
    u'b': 4.20439992,
    u'beta': 90.0,
    u'c': 4.20439992,
    u'gamma': 90.0,
    u'matrix': [[4.20439992, 0.0, 0.0],
     [0.0, 4.20439992, 0.0],
     [0.0, 0.0, 4.20439992]],
    u'volume': 74.32108777870904},
   u'sites': [{u'abc': [0.0, 0.5, 0.5],
     u'label': u'Al',
     u'properties': {},
     u'species': [{u'@class': u'Element',
       u'@module': u'pymatgen.core.periodic_table',
       u'element': u'Al',
       u'occu': 1}],
     u'xyz': [0.0, 2.10219996, 2.10219996]},
    {u'abc': [0.5, 0.5, 0.0],
     u'label': u'Al',
 

In [22]:
next(os.walk('./mapidoc/materials'))[1]

['anonymous_formula',
 'band_gap',
 'band_structure',
 'band_structure_uniform',
 'blessed_tasks',
 'bv_structure',
 'chemsys',
 'cif',
 'cifs',
 'cpu_time',
 'created_at',
 'decomposes_to',
 'delta_volume',
 'density',
 'dos',
 'e_above_hull',
 'efermi',
 'elasticity',
 'elements',
 'encut',
 'exp',
 'exp_lattice',
 'final_energy',
 'final_energy_per_atom',
 'formation_energy_per_atom',
 'full_formula',
 'has_bandstructure',
 'hubbards',
 'icsd_id',
 'icsd_ids',
 'initial_structure',
 'input',
 'ionic_steps',
 'is_compatible',
 'is_hubbard',
 'is_ordered',
 'magnetic_type',
 'nelements',
 'nkpts',
 'nsites',
 'ntask_ids',
 'original_task_id',
 'oxide_type',
 'pretty_formula',
 'pseudo_potential',
 'reduced_cell_formula',
 'run_stats',
 'run_type',
 'snl',
 'snl_final',
 'snlgroup_changed',
 'snlgroup_id',
 'snlgroup_id_final',
 'spacegroup',
 'static',
 'structure',
 'task_id',
 'task_ids',
 'total_magnetization',
 'unit_cell_formula',
 'updated_at',
 'volume',
 'warnings',
 'xrd']

In [27]:
len(element_systems)

25297

In [28]:
element_systems

['Ac-B',
 'Ac-B',
 'Ac-B',
 'Ac-B',
 'Ac-Cr',
 'Ac-Mo',
 'Ac-Mo',
 'Ac-Pt',
 'Ac-Pt',
 'Ac-Pt',
 'Ac-Pt',
 'Ac-Pt',
 'Ac-Pt',
 'Ac-Pt',
 'Ac-Pt',
 'Ac-Pt',
 'Ag-Al',
 'Ag-Al',
 'Ag-Al',
 'Ag-Al',
 'Ag-Al',
 'Ag-Al',
 'Ag-As',
 'Ag-As',
 'Ag-As',
 'Ag-B',
 'Ag-B',
 'Ag-Ba',
 'Ag-Ba',
 'Ag-Ba',
 'Ag-Ba',
 'Ag-Ba',
 'Ag-Ba',
 'Ag-Ba',
 'Ag-Be',
 'Ag-Be',
 'Ag-Be',
 'Ag-Be',
 'Ag-Bi',
 'Ag-C',
 'Ag-Ca',
 'Ag-Ca',
 'Ag-Ca',
 'Ag-Ca',
 'Ag-Ca',
 'Ag-Ca',
 'Ag-Ca',
 'Ag-Ca',
 'Ag-Ca',
 'Ag-Ca',
 'Ag-Cd',
 'Ag-Cd',
 'Ag-Cd',
 'Ag-Cd',
 'Ag-Cd',
 'Ag-Cd',
 'Ag-Cd',
 'Ag-Cd',
 'Ag-Cd',
 'Ag-Cd',
 'Ag-Ce',
 'Ag-Ce',
 'Ag-Ce',
 'Ag-Ce',
 'Ag-Ce',
 'Ag-Ce',
 'Ag-Ce',
 'Ag-Ce',
 'Ag-Ce',
 'Ag-Ce',
 'Ag-Ce',
 'Ag-Co',
 'Ag-Co',
 'Ag-Cr',
 'Ag-Cr',
 'Ag-Cu',
 'Ag-Dy',
 'Ag-Dy',
 'Ag-Dy',
 'Ag-Dy',
 'Ag-Dy',
 'Ag-Dy',
 'Ag-Dy',
 'Ag-Dy',
 'Ag-Er',
 'Ag-Er',
 'Ag-Er',
 'Ag-Er',
 'Ag-Er',
 'Ag-Er',
 'Ag-Er',
 'Ag-Eu',
 'Ag-Eu',
 'Ag-Eu',
 'Ag-Eu',
 'Ag-Eu',
 'Ag-Eu',
 'Ag-Eu',
 'Ag-Eu',
 'Ag-Eu',
 'Ag-Fe